<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
Notebook Imports
<a class="anchor" id="1"></a> 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
XGBoost Regression Model
<a class="anchor" id="1"></a> 

In [2]:
# Import NumPy and Pandas libraries

import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from scipy.stats import uniform

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# Import dataset
df = pd.read_csv('processedmath.csv')

# Make a copy of the dataframe
df_copy = df.copy()


In [3]:
dfX = df.iloc[:, 0:(len(df.columns)-4)] # Features
dfy = df.G3 # Target variable
print(dfX.shape)
print(type(dfX))
print(dfy.shape)
print(type(dfy))

(395, 32)
<class 'pandas.core.frame.DataFrame'>
(395,)
<class 'pandas.core.series.Series'>


In [4]:
# Extract the column names from the two datasets
col_names = list(dfX.columns)
col_names.append(dfy.name)
print(col_names)

['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']


In [5]:
# Conversion from Pandas Dataframe to NumPy Array
X = dfX.values
y = dfy.values
print(type(X))
print(X)

<class 'numpy.ndarray'>
[[ 0  0 18 ...  6  5  6]
 [ 0  0 17 ...  4  5  5]
 [ 0  0 15 ... 10  7  8]
 ...
 [ 1  1 21 ...  3 10  8]
 [ 1  1 18 ...  0 11 12]
 [ 1  1 19 ...  5  8  9]]


In [6]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=2)
#X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25)

In [7]:
xgbR = XGBRegressor(n_estimators = 20, learning_rate = 0.1)
xgbR.fit(X_train, y_train.ravel())
test_mse_before_tune =  mean_squared_error(xgbR.predict(X_test), y_test)
train_mse_before_tune = mean_squared_error(xgbR.predict(X_train), y_train)

print('***Original Model***')
print('the training mean squared error is: ', train_mse_before_tune)
print('the testing mean squared error is: ', test_mse_before_tune)

***Original Model***
the training mean squared error is:  2.594534222621698
the testing mean squared error is:  3.586443950543177


<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
K-Fold Cross Validation
<a class="anchor" id="1"></a> 

In [8]:
results = cross_validate(xgbR, X, y, scoring='neg_mean_squared_error', cv=5, return_train_score = True)
print('train score: ', results['train_score'])
print('test score: ', results['test_score'])

train score:  [-2.43194634 -2.56662208 -2.5436719  -2.64672987 -2.62403549]
test score:  [-5.76417453 -7.08426513 -4.4863901  -5.57538117 -5.65648791]


In [9]:
print('***Original Model***')
print('cross val training mean_squared_error is:', sum(-results['train_score'])/len(results['train_score']))
print('cross val testing mean_squared_error is:', sum(-results['test_score'])/len(results['test_score']))

***Original Model***
cross val training mean_squared_error is: 2.562601136674392
cross val testing mean_squared_error is: 5.713339766111601


<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
(Tunning) Statsmodel Ordinary Least Squares
<a class="anchor" id="1"></a> 

In [10]:
#The statsmodel.api provides key summary statistics
#The lm2 model refers to statsmodel.api model
#Add a constant to the X_train dataset and rename it as X_train_sm
X_train_sm = sm.add_constant(X_train)
print(X_train_sm)
print(X_train_sm.shape)

#Create another model using statsmodel.api using the Ordinary Least Squares (OLS) method for the model
lm2 = sm.OLS(y_train, X_train_sm).fit()

lm2.summary()

[[ 1.  0.  0. ...  0.  9.  9.]
 [ 1.  0.  0. ...  2. 10. 10.]
 [ 1.  1.  1. ...  3. 14. 12.]
 ...
 [ 1.  0.  0. ...  2.  8.  6.]
 [ 1.  0.  0. ...  4. 14. 14.]
 [ 1.  0.  0. ...  0.  6.  7.]]
(296, 33)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     42.35
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           1.39e-85
Time:                        23:04:39   Log-Likelihood:                -608.89
No. Observations:                 296   AIC:                             1284.
Df Residuals:                     263   BIC:                             1406.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8209      2.470     -0.332      0.740      -5.685       4.043
x1             0.6755      0.434      1.558      0.120      -0.178       1.529
x2             0.0347      0.284      0.122      0.903      -0.525       0.595
x3            -0.2335      0.119     -1.965      0.051      -0.468       0.001
x4             0.0361      0.345      0.105      0.917      -0.643       0.715
x5            -0.1970      0.283     -0.697      0.486      -0.754       0.360
x6             0.0669      0.395      0.169      0.866      -0.712       0.846
x7             0.1236      0.150      0.822      0.412      -0.172       0.420
x8             0.0224      0.152      0.147      0.883      -0.277       0.322
x9             0.0203      0.090      0.225      0.822      -0.158       0.198
x10            0.0461      0.103      0.447      0.655      -0.157       0.249
x11            0.1003      0.129      0.775      0.439      -0.155       0.355
x12           -0.2877      0.220     -1.305      0.193      -0.722       0.146
x13            0.1525      0.191      0.800      0.425      -0.223       0.528
x14           -0.1223      0.169     -0.724      0.469      -0.455       0.210
x15            0.2386      0.207      1.151      0.251      -0.170       0.647
x16            0.5433      0.381      1.427      0.155      -0.206       1.293
x17            0.0647      0.273      0.237      0.813      -0.472       0.602
x18            0.0567      0.274      0.207      0.836      -0.483       0.597
x19           -0.3451      0.251     -1.376      0.170      -0.839       0.149
x20           -0.4850      0.332     -1.460      0.145      -1.139       0.169
x21            0.1258      0.613      0.205      0.838      -1.082       1.333
x22           -0.0577      0.331     -0.174      0.862      -0.709       0.594
x23           -0.3177      0.265     -1.199      0.232      -0.839       0.204
x24            0.3168      0.138      2.299      0.022       0.046       0.588
x25            0.1032      0.130      0.792      0.429      -0.154       0.360
x26           -0.0074      0.132     -0.056      0.956      -0.268       0.253
x27           -0.1853      0.202     -0.918      0.359      -0.583       0.212
x28            0.1586      0.139      1.143      0.254      -0.114       0.432
x29            0.0903      0.087      1.038      0.300      -0.081       0.262
x30            0.0424      0.016      2.726      0.007       0.012       0.073
x31            0.2125      0.071      2.991      0.003       0.073       0.352
x32            0.9649      0.061     15.801      0.000       0.845       1.085
==============================================================================
Omnibus:                      129.071   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              487.604
Sk

In [11]:
#Using important features (P < 0.05) to evaluate the model
dfX_new = df[['famrel', 'absences', 'G1', 'G2']]
dfy_new = df.G3
col_names = list(dfX_new.columns)
col_names.append(dfy_new.name)
print(col_names)
X = dfX_new.values
y = dfy_new.values

X_train,X_test,y_train,y_test=train_test_split(X, y.ravel(), test_size=0.25, random_state=2)
#X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25)

['famrel', 'absences', 'G1', 'G2', 'G3']


In [12]:
xgbR_new = XGBRegressor(n_estimators = 20, learning_rate = 0.1)
xgbR_new.fit(X_train, y_train.ravel())

print('***New Model***')
print('Training MSE is: ',  mean_squared_error(xgbR_new.predict(X_train), y_train))
print('Testing MSE is: ', mean_squared_error(xgbR_new.predict(X_test), y_test))

***New Model***
Training MSE is:  3.6651904351041913
Testing MSE is:  3.7391792226623606


<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
Grid Search
<a class="anchor" id="1"></a> 

In [13]:
mean_squared_error(xgbR_new.predict(X_test), y_test)
#DTR = DecisionTreeRegressor(max_depth = 4, min_samples_leaf =1)
xgbR_new = XGBRegressor()
param_grid = {
    "n_estimators": range (10,100),
    "learning_rate": [0.01, 0.03, 0.05, 0.09, 0.1, 0.2],
    "max_depth": [2,4,6,8]
}

gs = GridSearchCV(xgbR_new, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(-gs.best_score_)
print(gs.best_params_)

2.9543059058966543
{'learning_rate': 0.09, 'max_depth': 2, 'n_estimators': 62}


In [14]:
xgbR_final = XGBRegressor(n_estimators = 62,
                          learning_rate = 0.09,
                          max_depth = 2)

xgbR_final.fit(X_train, y_train.ravel())

preds = xgbR_final.predict(X_test)

print('***Final Model***')
print('Training MSE after tuning is: ', mean_squared_error(xgbR_final.predict(X_train), y_train))
print('Testing MSE after tuning is: ', mean_squared_error(xgbR_final.predict(X_test), y_test))
print('The testing MSE has improved by ', train_mse_before_tune-mean_squared_error(xgbR_final.predict(X_train), y_train), 'after tuning.')
print('The testing MSE has improved by ', test_mse_before_tune-mean_squared_error(xgbR_final.predict(X_test), y_test), 'after tuning.')

***Final Model***
Training MSE after tuning is:  2.4216796033716963
Testing MSE after tuning is:  1.3089113450017726
The testing MSE has improved by  0.17285461925000156 after tuning.
The testing MSE has improved by  2.2775326055414045 after tuning.


<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
K-Folds Cross Validation (Final Model)
<a class="anchor" id="1"></a> 

In [15]:
results = cross_validate(xgbR_final, X, y.ravel(), scoring='neg_mean_squared_error', cv=5, return_train_score = True)
print('train score: ', results['train_score'])
print('test score: ', results['test_score'])

train score:  [-2.06785415 -2.15221592 -2.23682383 -1.99879505 -1.79611124]
test score:  [-2.66085946 -2.36243301 -1.74940083 -3.51992089 -4.43500323]


In [16]:
print('***Final Model***')
print('cross val training mean_squared_error is:', sum(-results['train_score'])/len(results['train_score']))
print('cross val testing mean_squared_error is:', sum(-results['test_score'])/len(results['test_score']))

***Final Model***
cross val training mean_squared_error is: 2.0503600409315585
cross val testing mean_squared_error is: 2.945523483861085
